In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
cases_ = pd.read_csv('https://raw.githubusercontent.com/idandrd/israel-covid19-data/master/IsraelCOVID19.csv')
# some pesky unicode chars in column names
cases_ = cases_.rename(columns={'בינוני Moderate' : 'Moderate', 'קשה Severe': 'Severe', 'החלימו Recovered': 'Recovered', 'נפטרו Deceased' : 'Deceased'})
cases_.rename(columns = lambda x : x.encode('ascii', 'ignore').decode('ascii'), inplace=True)

In [ ]:
# convert to numeric
#cases = cases_.apply(lambda x: pd.to_numeric(x, errors='coerce'), axis=1)
cases = cases_

cases['Date'] = pd.to_datetime(cases['Date'])
cases.index = cases['Date']

cases['tot'] = cases['New Cases'].cumsum()
new = cases[['Recovered','Severe', 'Deceased']].diff()
cases = cases.merge(new, left_index=True, right_index=True, suffixes = ('',' New'))

cases['added'] = cases['Severe'] + cases['Deceased']
cases['added_rel3'] = 1+cases['added'].rolling(3).mean().pct_change()
cases['doubling rate'] = np.log(2) / np.log(cases['added_rel3'])


In [ ]:
plt.figure()
#pl.xaxis.set_major_locator(plt.MaxNLocator(5))
pl=cases['added_rel3'].plot(linewidth=3)
pl.set_ylim((0.95, 1.65))
plt.xticks(rotation=270)
pl.set_xlim(left=pd.to_datetime('2020-03-16'), right=cases.index[-1])
#pl.locator_params(axis='x', nbins=15)
#pl.xaxis.set_ticks(np.arange(0, len(cases.index), 4))
#pl.set_xticks(cases.index)
plt.show()

In [ ]:
cases['new added'] = (cases['Severe New'] + cases['Deceased New']).rolling(3).mean()
plt.figure()
c = cases[-21:]
pl=c['new added'].plot(linewidth=3)
#plt.xticks(rotation=270)
pl.set_xlim(left=c.index[0], right=c.index[-1])
plt.show()